# QuickStart
Based on [Pytorch Quickstart](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

- This section runs through the API for common tasks in ML.

## Working with data

PyTorch has 2 primitives to work with data: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`.

`Dataset` stores the samples and their corresponding labels, and `Dataloader` wraps an iterable around the `Dataset`.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch has TorchText, TorchVision and TorchAudio, all contain datasets.

We will use a TorchVision dataset.

The `torchvision.datasets` module contains `Dataset` objects for many real-world vision data like CIFAR, COCO etc

Here is the [full list](https://pytorch.org/vision/stable/datasets.html)

We'll use a 3 datasets to get the gist: FashionMNIST.

Every TorchVision Dataset includes two arguments: `transform` and `target_transform` to modify the samples and labels respectively.

In [2]:
# download training data from open datasets

train_data = datasets.FashionMNIST(
root = 'data',
download=True,
train = True,
transform=ToTensor()
) #required argument = `root` if locally available else #2nd required argument is download=True 

test_data = datasets.FashionMNIST(
root = 'data',
download=True,
train = False,
transform=ToTensor()
) #required argument = `root` if locally available else #2nd required argument is download=True 

100%|██████████| 26421880/26421880 [00:00<00:00, 114678849.43it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 4997976.61it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 62410461.08it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 8703054.01it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



The `Dataset` is passed as an argument to `DataLoader` to wrap an iterable over our dataset and support automatic batching, sampling, shuffling and multiprocess data loading.

A batch size of 64 means each element in the dataloader iterable will return a batch of 64 features & labels.

In [3]:
#Create dataloaders.
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} & data type : {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) & data type : torch.int64


In [4]:
train_dataloader.dataset.classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [5]:
classes = train_dataloader.dataset.classes
print(type(classes))
print(classes)

<class 'list'>
['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


# Creating Models

To define a NN in PyTorch, a class is created that inherits from nn. Module, the layers are defined in the network in the __init__ function and specify how data will pass through the network in the forward function. To accelerate in the NN, we move it to th GPU or MPS if available

In [6]:
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backend.mps.is_available() else "cpu")
print("Device : ", device)

Device :  cuda


In [7]:
#Define Model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28,512),nn.ReLU(),nn.Linear(512,512),nn.ReLU(),nn.Linear(512,10))
    
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

        

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# optimizing the Model Parameters
to train a model a loss function and an optimizer are used.

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches) and backpropagates the prediction error to adjust the model's parameters

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() #Why?
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        #compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)
        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()#why?
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

# Check the model's performance against the test dataset to ensure it is learning
def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Average Loss : {test_loss:>8f} \n")

# train over epochs

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n --------------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!!!")

Epoch 1
 --------------------------------------
loss: 2.311084 [   64/60000]
loss: 0.537839 [ 6464/60000]
loss: 0.389876 [12864/60000]
loss: 0.515429 [19264/60000]
loss: 0.466273 [25664/60000]
loss: 0.452351 [32064/60000]
loss: 0.372711 [38464/60000]
loss: 0.514988 [44864/60000]
loss: 0.526308 [51264/60000]
loss: 0.488072 [57664/60000]
Test Error: 
 Accuracy: 84.2%, Average Loss : 0.431486 

Epoch 2
 --------------------------------------
loss: 0.290335 [   64/60000]
loss: 0.351618 [ 6464/60000]
loss: 0.306716 [12864/60000]
loss: 0.408244 [19264/60000]
loss: 0.422910 [25664/60000]
loss: 0.391173 [32064/60000]
loss: 0.308097 [38464/60000]
loss: 0.474651 [44864/60000]
loss: 0.393032 [51264/60000]
loss: 0.411235 [57664/60000]
Test Error: 
 Accuracy: 85.2%, Average Loss : 0.399621 

Epoch 3
 --------------------------------------
loss: 0.215236 [   64/60000]
loss: 0.340540 [ 6464/60000]
loss: 0.222811 [12864/60000]
loss: 0.321803 [19264/60000]
loss: 0.366737 [25664/60000]
loss: 0.379949 [3

In [11]:
#model classes?


# Saving Models

A common way to save models is to serialize the internal state dictionary that contains the model paramters. (Simply saving the weights).

Here we'll save the classes too.

In [12]:
torch.save(model.state_dict(),"model_weights.pth") #save weights only
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [13]:
#save weights and classes
torch.save({'model_state':model.state_dict(),'classes':classes},"model_weights_and_cls.pth") #save weights
print("Saved PyTorch Model State and classes to model_weights_and_cls.pth")

Saved PyTorch Model State and classes to model_weights_and_cls.pth


# Loading Models

The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [14]:
#weights only
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model_weights.pth"))

<All keys matched successfully>

In [15]:
#weights and classes
model = NeuralNetwork().to(device)
checkpoint = torch.load("model_weights_and_cls.pth")
print(checkpoint.keys())

dict_keys(['model_state', 'classes'])


In [16]:
model.load_state_dict(checkpoint['model_state'])
classes = checkpoint['classes']
print(classes)

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


The model can now be used for predictions

In [17]:
# Get the list of all the classes (target values)
classes = checkpoint['classes']
print('classes :',classes)


classes : ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [18]:
model.train()#why?

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [19]:
model.eval() #why? ## This disables dropout and batch normalization layers,
#ensuring consistent outputs during evaluation


x,y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted} \n Actual: {actual}")

Predicted: Ankle boot 
 Actual: Ankle boot


In [20]:
# len(train_dataloader)

In [21]:
# train_dataloader.__len__() #number of batches

In [22]:
# len(train_dataloader.dataset) #total number of samples

In [23]:
# dummy_train_data = datasets.CIFAR10(root='data1',download=True)

In [24]:
# dummy_train_data = datasets.CIFAR100(root='data1',download=True)

In [25]:
# dummy_train_data = datasets.Country211(root='data1',download=True)

In [26]:
# dummy_train_data = datasets.DTD(root='data1',download=True)